In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from pybaseball import statcast, cache
#from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import joblib
import math
from pybaseball import statcast
import scipy.stats as stats
#from catboost import Pool
#import optuna
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score, precision_score
import sqlite3
cache.enable()

In [2]:
data = pd.read_csv('savant_data.csv')

In [3]:
data['velocity'].mean()

np.float64(87.48719346049047)

In [4]:
# First, let's select the key features for the "Stuff+" model as discussed:
# - 'pitch_type'
# - 'pitch_percent'
# - 'swing_miss_percent'
# - 'arm_angle'
# - 'xbadiff'
# - 'xobp'
# - 'xslg'
# - 'pitcher_run_value_per_100'
# - 'batter_run_value_per_100'

# We will also select 'swing_miss_percent' as our initial target variable

# Selecting the relevant features
selected_columns = [
    'pitch_type', 'pitch_percent', 'swing_miss_percent', 'arm_angle', 
    'xbadiff', 'xobp', 'xslg', 'pitcher_run_value_per_100', 'batter_run_value_per_100'
]

data = data.dropna(subset=selected_columns)

# Extracting the features and target variable
features = data[selected_columns]
target = data['swing_miss_percent']

# One-hot encoding the categorical 'pitch_type' column to handle it in the model
features_encoded = pd.get_dummies(features, columns=['pitch_type'], drop_first=True)

# Splitting the data into training and testing sets for modeling
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_encoded, target, test_size=0.2, random_state=42)

# We will now train an initial regression model. I'll start with a Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize the model
rf_model = RandomForestRegressor(random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Calculate the mean squared error for performance evaluation
mse = mean_squared_error(y_test, y_pred)
mse


np.float64(0.0019141906923948483)

In [5]:
comparison = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

print(comparison)

      Actual  Predicted
3116    31.7     31.700
668     37.5     37.500
1213    36.4     36.400
478     55.0     55.045
530     30.5     30.500
...      ...        ...
4197    22.4     22.400
321     44.4     44.366
3043     8.3      8.325
888     40.0     39.999
655     43.1     43.089

[881 rows x 2 columns]


In [6]:
'''
data['SwingMiss_norm'] = (data['swing_miss_percent'] / data['swing_miss_percent'].mean()) * 100
data['xwOBA_norm'] = ((data['xwoba'] / data['xwoba'].mean())) * 100 # Inverse
data['RunExp_norm'] = ((data['run_exp'] / data['run_exp'].mean())) * 100  # Inverse

# Assigning weights
w1, w2, w3 = .3, .2, .5

# Calculating Stuff+
data['Stuff+'] = (
    w1 * data['SwingMiss_norm'] +
    w2 * data['xwOBA_norm'] +
    w3 * data['RunExp_norm']
)

# Display the first few rows of the Stuff+ calculation
data[['swing_miss_percent', 'xobp', 'pitcher_run_value_per_100', 'Stuff+']].head()
'''

"\ndata['SwingMiss_norm'] = (data['swing_miss_percent'] / data['swing_miss_percent'].mean()) * 100\ndata['xwOBA_norm'] = ((data['xwoba'] / data['xwoba'].mean())) * 100 # Inverse\ndata['RunExp_norm'] = ((data['run_exp'] / data['run_exp'].mean())) * 100  # Inverse\n\n# Assigning weights\nw1, w2, w3 = .3, .2, .5\n\n# Calculating Stuff+\ndata['Stuff+'] = (\n    w1 * data['SwingMiss_norm'] +\n    w2 * data['xwOBA_norm'] +\n    w3 * data['RunExp_norm']\n)\n\n# Display the first few rows of the Stuff+ calculation\ndata[['swing_miss_percent', 'xobp', 'pitcher_run_value_per_100', 'Stuff+']].head()\n"

In [7]:
print(data['xwoba'].min())
print(data['xwoba'].max())
print(data['xwoba'].mean())
print(data['pitcher_run_exp'].min())
print(data['pitcher_run_exp'].max())
print(data['pitcher_run_exp'].mean())


0.0
1.861
0.32408764759309716
-53.1
79.3
0.24831970935513178


In [8]:
# different normailzation so all are around 60-150, average at 100
data['pre_new_all_+'] = data['pitcher_run_exp'] - data['pitcher_run_exp'].min()
print(data['pre_new_all_+'].mean())

53.34831970935513


In [194]:
data = pd.read_csv('savant_data.csv')

std_dev_pre = data['pitcher_run_exp'].std()
std_dev_xwoba = data['xwoba'].std()
std_dev_swm = data['swing_miss_percent'].std()
#std_dev_hhr = data['hardhit_percent'].std()
#std_dev_prve100 = data['pitcher_run_value_per_100'].std()

# try with standard dev
data['SwingMiss_norm'] = ((data['swing_miss_percent'] - data['swing_miss_percent'].mean()) / std_dev_swm) * 10
data['xwOBA_norm'] = (((data['xwoba'].mean() - data['xwoba'])) / std_dev_xwoba) * 10
data['RunExp_norm'] = ((data['pitcher_run_exp'] - data['pitcher_run_exp'].mean()) / std_dev_pre) * 10
#data['HardHit_norm'] = ((data['hardhit_percent'].mean() - data['hardhit_percent']) / std_dev_hhr) * 10
#data['prve100_norm'] = ((data['pitcher_run_value_per_100'] - data['pitcher_run_value_per_100'].mean()) / std_dev_prve100) * 50

# Assigning weights
w1, w2, w3 = 1/3, 1/3, 1/3

# Calculating Stuff+
data['Stuff+'] = (
    w1 * data['SwingMiss_norm'] +
    w2 * data['xwOBA_norm'] +
    w3 * data['RunExp_norm']
)
#w4 * data['HardHit_norm']

data['Stuff+'] = (data['Stuff+']) + 100

# Display the first few rows of the Stuff+ calculation
data[['swing_miss_percent', 'xwoba', 'pitcher_run_exp', 'hardhit_percent', 'Stuff+']].head()

,swing_miss_percent,xwoba,pitcher_run_exp,hardhit_percent,Stuff+
0,44.9,0.225,72.7,32.053176,134.864968
1,39.5,0.243,60.0,29.984544,127.835927
2,56.7,0.107,14.5,26.785714,126.447414
3,58.4,0.146,19.0,23.076923,126.400194
4,25.7,0.300,79.3,34.270650,126.163772


In [195]:
features_sorted = data.sort_values(by='Stuff+', ascending=False)
print(data['Stuff+'].mean())
features_sorted.to_csv('savant_data.csv', index=False)

100.0


In [196]:
filtered_data = data[data['pitches'] >= 200]
filtered_data = filtered_data.sort_values(by='Stuff+', ascending=False)
filtered_data.to_csv('STUFF+Initial.csv', index=False)

In [197]:
print(filtered_data[filtered_data['pitch_type'] == 'FC'].head())

    pitches  player_id         player_name  total_pitches pitch_type  \
4      6224     669203      Burnes, Corbin          11943         FC   
11     1278     621237      Alvarado, José           3720         FC   
16      318     596112  Stephenson, Robert           2444         FC   
59     2802     661403     Clase, Emmanuel           4080         FC   
88     1489     677651        Garcia, Luis           5435         FC   

    pitch_percent     ba    iso  babip    slg  ...  z_diff  fastball_x_y  \
4            52.1  0.227  0.115  0.278  0.341  ...   -0.07          0.69   
11           34.4  0.136  0.063  0.327  0.199  ...   -0.01          0.80   
16           13.0  0.101  0.152  0.143  0.253  ...    0.21          1.15   
59           68.7  0.212  0.071  0.259  0.283  ...   -0.02          0.69   
88           27.4  0.167  0.119  0.237  0.286  ...    0.04          1.50   

    fastball_z_y  fastball_x  fastball_z  HardHit_norm  veloXmovement  \
4           6.01        0.69        6

In [247]:
# Stuff Model!
# First, let's select the key features for the "Stuff+" model as discussed:
# - 'pitch_type'
# - 'velocity'
# - 'api_break_z_induced'
# - 'api_break_x_arm'
# - 'release_extension'
# - 'spin_rate'
# - 'release_pos_x'
# - 'release_pos_z'
# - 'x_diff'  
# - 'z_diff'
# - 'Stuff+'

data = pd.read_csv('STUFF+Initial.csv')

# Adding New Variables, interactions
data['veloXspin'] = data['velocity'] * data['spin_rate']
data['veloXmovement'] = data['velocity'] * data['api_break_z_induced'] * data['api_break_x_arm']
data['vertXhoriz'] = data['api_break_z_induced'] * data['api_break_x_arm']
data['veloXrelease'] = data['release_extension'] * data['velocity']

# Selecting the relevant features
selected_columns = [
    'pitch_type', 'velocity', 'api_break_z_induced', 'api_break_x_arm', 'release_extension', 'spin_rate', 'release_pos_x_norm', 'release_pos_z', 'x_diff', 'z_diff', 'veloXrelease', 'veloXmovement' 
]

data = data.dropna(subset=selected_columns)

# Extracting the features and target variable
features = data[selected_columns]
target = data['Stuff+']

# One-hot encoding the categorical 'pitch_type' column to handle it in the model
features_encoded = pd.get_dummies(features, columns=['pitch_type'], drop_first=True)

# Splitting the data into training and testing sets for modeling
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_encoded, target, test_size=0.2, random_state=42)

# We will now train an initial regression model. I'll start with a Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize the model
rf_model = RandomForestRegressor(max_depth=20, max_features='sqrt', min_samples_leaf=2,
                      min_samples_split=10, n_estimators=300, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Calculate the mean squared error for performance evaluation
mse = mean_squared_error(y_test, y_pred)
mse


np.float64(38.86911871125941)

In [249]:
print(features_encoded.columns)
comparison = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

print(comparison)
print(y_test.max())
print(y_pred.max())
print(y_test.mean())
print(y_pred.mean())
print(y_test.min())
print(y_pred.min())


Index(['velocity', 'api_break_z_induced', 'api_break_x_arm',
       'release_extension', 'spin_rate', 'release_pos_x_norm', 'release_pos_z',
       'x_diff', 'z_diff', 'veloXrelease', 'veloXmovement', 'pitch_type_CU',
       'pitch_type_FC', 'pitch_type_FF', 'pitch_type_FO', 'pitch_type_FS',
       'pitch_type_KC', 'pitch_type_KN', 'pitch_type_SC', 'pitch_type_SI',
       'pitch_type_SL', 'pitch_type_ST', 'pitch_type_SV'],
      dtype='object')
          Actual   Predicted
1587   98.483882  105.535016
2203   93.911958   98.922799
772   104.658229  102.781919
1732   97.485941  103.792992
387   108.734389  101.085303
...          ...         ...
1905   96.233090  103.854915
252   110.997915  100.124339
1793   96.985262  101.595304
2332   92.833100   96.056152
2103   94.695396  103.813364

[573 rows x 2 columns]
121.266536736447
111.85211189810039
99.9140551925534
100.29518882277591
80.63239245783055
89.53508902240219


In [250]:
#pitch_data = [data['velocity'], data['api_break_z_induced'], data['api_break_x_arm'], data['release_extension'], data['spin_rate'], data['release_pos_x_norm'], data['release_pos_z'], data['x_diff'], data['z_diff'],  data['pitch_type_CU'], data['pitch_type_FC'], data['pitch_type_FF'], data['pitch_type_FO'], data['pitch_type_FS'], data['pitch_type_KC'], data['pitch_type_KN'], data['pitch_type_SC'], data['pitch_type_SI'], data['pitch_type_SL'], data['pitch_type_ST'], data['pitch_type_SV']]
predictions = rf_model.predict(features_encoded)
data['Stuff+_perdicted'] = predictions
data['Stuff_diff'] = data['Stuff+'] - data['Stuff+_perdicted']
data['NewStuff+_perdicted'] = ((data['Stuff+_perdicted'] - 100) * 2) + 100
data['NewStuff_diff'] = data['Stuff+'] - data['NewStuff+_perdicted']
filtered_data = data[data['pitches'] >= 200]
filtered_data = filtered_data.sort_values(by='Stuff+', ascending=False)
filtered_data.to_csv('STUFF+Initial.csv', index=False)
mse = mean_squared_error(filtered_data['Stuff+'], filtered_data['NewStuff+_perdicted'])
mse

np.float64(32.32139961824945)

In [251]:

my_fastball = [98, 1.4, 1.2, 6.5, 2300, 2.55, 6.02, 0, 0, False, False, False, True, False, False, False, False, False, False, False, False, False]
#a_slider = rf_model.predict([my_slider], columns=features_encoded)

import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names")
a_fastball = rf_model.predict([my_fastball])
print(a_fastball)

ValueError: X has 22 features, but RandomForestRegressor is expecting 23 features as input.

In [232]:
data = pd.read_csv('savant_data.csv')

std_dev_ba = data['ba'].std()
std_dev_ls = data['launch_speed'].std()
std_dev_swm = data['swing_miss_percent'].std()

# try with standard dev
data['SwingMiss_norm'] = ((data['swing_miss_percent'] - data['swing_miss_percent'].mean()) / std_dev_swm) * 10
data['ls_norm'] = ((data['launch_speed'].mean() - data['launch_speed']) / std_dev_ls) * 10
data['ba_norm'] = ((data['ba'].mean() - data['ba']) / std_dev_ba) * 10

# Assigning weights
w1, w2, w3 = 1/3, 1/3, 1/3

# Calculating Unhittable+
data['Unhittable+'] = (
    w1 * data['SwingMiss_norm'] +
    w2 * data['ls_norm'] +
    w3 * data['ba_norm']
)

data['Unhittable+'] = (data['Unhittable+'] + 100)

# Display the first few rows of the Stuff+ calculation
data[['swing_miss_percent', 'ba', 'launch_speed', 'Unhittable+']].head()

,swing_miss_percent,ba,launch_speed,Unhittable+
0,44.9,0.169,86.3,112.741857
1,39.5,0.169,85.6,111.602048
2,56.7,0.102,84.6,122.675825
3,58.4,0.099,79.9,128.610712
4,25.7,0.227,87.8,100.936010


In [233]:
filtered_data_unhit = data[data['pitches'] >= 200]
filtered_data_unhit = filtered_data_unhit.sort_values(by='Unhittable+', ascending=False)
filtered_data_unhit.to_csv('Unhittable+Initial.csv', index=False)
data['Unhittable+'].mean()

np.float64(100.0)

In [234]:
'''
data['release_pos_x_norm'] = data['release_pos_x']
data['release_pos_x_norm'] = data['release_pos_x_norm'].apply(lambda x: -x if x < 0 else x)

# Filter for fastballs
fastball_df = data[data['pitch_type'] == 'FF']

# Group by pitcher and calculate average x_pos and z_pos for fastballs
fastball_positions = fastball_df.groupby('player_name')[['release_pos_x_norm', 'release_pos_z']].mean().reset_index()
fastball_positions.rename(columns={'release_pos_x_norm': 'fastball_x', 'release_pos_z': 'fastball_z'}, inplace=True)

# Step 2: Merge fastball positions into the original dataset
data = data.merge(fastball_positions, on='player_name', how='left')

# Step 3: Calculate differences for x_pos and z_pos
data['x_diff'] = data['release_pos_x_norm'] - data['fastball_x']
data['z_diff'] = data['release_pos_z'] - data['fastball_z']

# Step 4: Filter for non-fastball pitches
secondary_pitches = data[data['pitch_type'] != 'FF']

# Step 5: Keep only relevant columns for the result
result = secondary_pitches[['player_name', 'pitch_type', 'x_diff', 'z_diff']]

# Output the result
print(result.head())
'''

"\ndata['release_pos_x_norm'] = data['release_pos_x']\ndata['release_pos_x_norm'] = data['release_pos_x_norm'].apply(lambda x: -x if x < 0 else x)\n\n# Filter for fastballs\nfastball_df = data[data['pitch_type'] == 'FF']\n\n# Group by pitcher and calculate average x_pos and z_pos for fastballs\nfastball_positions = fastball_df.groupby('player_name')[['release_pos_x_norm', 'release_pos_z']].mean().reset_index()\nfastball_positions.rename(columns={'release_pos_x_norm': 'fastball_x', 'release_pos_z': 'fastball_z'}, inplace=True)\n\n# Step 2: Merge fastball positions into the original dataset\ndata = data.merge(fastball_positions, on='player_name', how='left')\n\n# Step 3: Calculate differences for x_pos and z_pos\ndata['x_diff'] = data['release_pos_x_norm'] - data['fastball_x']\ndata['z_diff'] = data['release_pos_z'] - data['fastball_z']\n\n# Step 4: Filter for non-fastball pitches\nsecondary_pitches = data[data['pitch_type'] != 'FF']\n\n# Step 5: Keep only relevant columns for the result

In [235]:
print(data['release_pos_x_norm'].mean())
print(data[data['player_name'] == 'Valdez, Framber']['release_pos_x_norm'].mean())

1.8892904578818597
0.85


In [236]:
"""
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 'log2', None]
}

rf_model = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train, y_train)

# Best parameters and model
best_params = grid_search.best_params_
best_rf_model = grid_search.best_estimator_
"""

"\nfrom sklearn.model_selection import GridSearchCV\n\nparam_grid = {\n    'n_estimators': [100, 200, 300],\n    'max_depth': [10, 20, None],\n    'min_samples_split': [2, 5, 10],\n    'min_samples_leaf': [1, 2, 5],\n    'max_features': ['sqrt', 'log2', None]\n}\n\nrf_model = RandomForestRegressor(random_state=42)\ngrid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2)\ngrid_search.fit(X_train, y_train)\n\n# Best parameters and model\nbest_params = grid_search.best_params_\nbest_rf_model = grid_search.best_estimator_\n"

In [237]:

print(best_params)
print(best_rf_model)


{'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 300}
RandomForestRegressor(max_depth=20, max_features='sqrt', min_samples_leaf=2,
                      min_samples_split=10, n_estimators=300, random_state=42)


In [238]:
# Importing XGBoost
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

data = pd.read_csv('STUFF+Initial.csv')

# Initialize the XGBoost model
xgb_model = XGBRegressor(
    random_state=42,  # Ensure reproducibility
    n_estimators=300,  # Number of trees (default: 100)
    learning_rate=0.01,  # Step size shrinkage (default: 0.3)
    max_depth=4,       # Maximum tree depth (default: 6)
    objective='reg:squarederror'  # Regression objective
)
# Train the XGBoost model
xgb_model.fit(X_train, y_train)

# Predict on the test set
y_pred = xgb_model.predict(X_test)

# Calculate the mean squared error for performance evaluation
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 39.764355157753606


In [239]:
comparison = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

print(y_test.max())
print(y_pred.max())
print(y_test.mean())
print(y_pred.mean())
print(y_test.min())
print(y_pred.min())

121.266536736447
110.30937
99.9140551925534
100.25163
80.63239245783055
91.44656


In [240]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [4, 6, 8]
}

# Initialize the model
xgb_model = XGBRegressor(random_state=42, objective='reg:squarederror')

# Perform Grid Search
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train, y_train)

# Get the best parameters and model
best_params = grid_search.best_params_
print(f"Best Parameters: {best_params}")

best_xgb_model = grid_search.best_estimator_


Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=200; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=200; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=200; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=300; total time=   0.3s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=300; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=300; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=6, n_es

In [241]:
print(data.head())

   pitches  player_id      player_name  total_pitches pitch_type  \
0     4861     656302     Cease, Dylan          12527         SL   
1     4610     450203  Morton, Charlie          11523         CU   
2      508     642585  Bautista, Félix           1983         FS   
3      955     518585   Cruz, Fernando           2518         FS   
4     6224     669203   Burnes, Corbin          11943         FC   

   pitch_percent     ba    iso  babip    slg  ...  fastball_z  HardHit_norm  \
0           38.8  0.169  0.098  0.262  0.267  ...        6.32      3.743808   
1           40.0  0.169  0.098  0.267  0.267  ...        5.50      5.111614   
2           25.6  0.102  0.072  0.264  0.175  ...        6.99      7.226722   
3           37.9  0.099  0.046  0.329  0.145  ...        5.95      9.679022   
4           52.1  0.227  0.115  0.278  0.341  ...        6.01      2.277585   

   veloXmovement  vertXhoriz  veloXrelease  prve100_norm  Stuff+_perdicted  \
0      -2.239677   -0.025773       536